In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
file_path = '/content/drive/MyDrive/Colab_Notebooks/Kaggle_Competition/Titanic - Machine Learning from Disaster/'

Mounted at /content/drive


In [3]:
train_data = pd.read_csv(file_path + 'train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data = pd.read_csv(file_path + 'test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [139]:
file_name = ['train.csv', 'test.csv']
file_names = []
for name in file_name:
  file_names.append([name, pd.read_csv(file_path + name)])


In [140]:
from uuid import NAMESPACE_OID
from sklearn.preprocessing import LabelEncoder

file_names_drop = []
for i, [name, df] in enumerate(file_names):
  print('★',name)

  df = df.drop(columns=['Name','Cabin'])
  # text to label
  Sex_encoder = LabelEncoder()
  df['Sex'] = Sex_encoder.fit_transform(df['Sex'])
  Embarked_encoder = LabelEncoder()
  df['Embarked'] = Embarked_encoder.fit_transform(df['Embarked'])
  Ticket_encoder = LabelEncoder()
  df['Ticket'] = Ticket_encoder.fit_transform(df['Ticket'])
  file_names_drop.append([name, df])
  '''
  # reverse label
  df['Sex'] = Sex_encoder.inverse_transform(df['Sex'])
  df['Embarked'] = Embarked_encoder.inverse_transform(df['Embarked'])
  #print(df.head())
  '''
  print(name, df.head())

★ train.csv
train.csv    PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch  Ticket     Fare  \
0            1         0       3    1  22.0      1      0     523   7.2500   
1            2         1       1    0  38.0      1      0     596  71.2833   
2            3         1       3    0  26.0      0      0     669   7.9250   
3            4         1       1    0  35.0      1      0      49  53.1000   
4            5         0       3    1  35.0      0      0     472   8.0500   

   Embarked  
0         2  
1         0  
2         2  
3         2  
4         2  
★ test.csv
test.csv    PassengerId  Pclass  Sex   Age  SibSp  Parch  Ticket     Fare  Embarked
0          892       3    1  34.5      0      0     152   7.8292         1
1          893       3    0  47.0      1      0     221   7.0000         2
2          894       2    1  62.0      0      0      73   9.6875         1
3          895       3    1  27.0      0      0     147   8.6625         2
4          896       3    0  2

In [141]:
for i, [name, df] in enumerate(file_names_drop):
  print('★',name)
  print(df.columns)
  df[df.isnull().any(axis=1)]
  df[['Age']] = df[['Age']].fillna(0)

  mean = df['Fare'].mean()
  df[['Fare']] = df[['Fare']].fillna(mean)
  print(df.isnull().sum())
  file_names_drop[i] = (name, df)
  print(name, df.head())

★ train.csv
Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Embarked'],
      dtype='object')
PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
train.csv    PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch  Ticket     Fare  \
0            1         0       3    1  22.0      1      0     523   7.2500   
1            2         1       1    0  38.0      1      0     596  71.2833   
2            3         1       3    0  26.0      0      0     669   7.9250   
3            4         1       1    0  35.0      1      0      49  53.1000   
4            5         0       3    1  35.0      0      0     472   8.0500   

   Embarked  
0         2  
1         0  
2         2  
3         2  
4         2  
★ test.csv
Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Embar

In [ ]:
train_data = file_names_drop[0][1]

corrcoef = train_data.corr()
corrcoef

In [142]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

# 特徵處理
# 讀取資料
train_data = file_names_drop[0][1]
test_data = file_names_drop[1][1]
submission_data = pd.read_csv(file_path + 'submission.csv')

# select_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']
# 1
scaler = MinMaxScaler()
train_data['Fare'] = scaler.fit_transform(train_data[['Fare']])
test_data['Fare'] = scaler.fit_transform(test_data[['Fare']])


# 2
select_features = ['Sex', 'Age']
train_data['Sex_Age'] = train_data['Sex'].astype(str) + '_' + train_data['Age'].astype(str)
test_data['Sex_Age'] = test_data['Sex'].astype(str) + '_' + test_data['Age'].astype(str)

# 3
select_features = ['Age', 'Fare']
train_data['Age_Fare'] = train_data['Age'] * train_data['Fare']
test_data['Age_Fare'] = test_data['Age'] * test_data['Fare']
train_data['Age_Fare_sum'] = train_data['Age'] + train_data['Fare']
test_data['Age_Fare_sum'] = test_data['Age'] + test_data['Fare']

# 4
train_data['Ticket_Count'] = train_data.groupby('Ticket')['Ticket'].transform('count')
test_data['Ticket_Count'] = test_data.groupby('Ticket')['Ticket'].transform('count')

# 5
select_features = ['Pclass', 'SibSp', 'Parch', 'Ticket']
train_data['Family'] = train_data['Pclass'] * train_data['Ticket'] * train_data['SibSp'] * train_data['Parch']
test_data['Family'] = test_data['Pclass'] * test_data['Ticket'] * test_data['SibSp'] * test_data['Parch']

# 6
train_data['Fare_Embarked'] = train_data['Fare'] * train_data['Embarked'].map({'C': 1, 'Q': 2, 'S': 3})
test_data['Fare_Embarked'] = test_data['Fare'] * test_data['Embarked'].map({'C': 1, 'Q': 2, 'S': 3})

print(train_data.head())
print(test_data.head())
print(train_data.columns)

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch  Ticket     Fare  \
0            1         0       3    1  22.0      1      0     523   7.2500   
1            2         1       1    0  38.0      1      0     596  71.2833   
2            3         1       3    0  26.0      0      0     669   7.9250   
3            4         1       1    0  35.0      1      0      49  53.1000   
4            5         0       3    1  35.0      0      0     472   8.0500   

   Embarked  Family  Sex_Age   Age_Fare  Ticket_Count  
0         2       0     22.0   159.5000             1  
1         0       0      0.0  2708.7654             1  
2         2       0      0.0   206.0500             1  
3         2       0      0.0  1858.5000             2  
4         2       0     35.0   281.7500             1  
   PassengerId  Pclass  Sex   Age  SibSp  Parch  Ticket     Fare  Embarked  \
0          892       3    1  34.5      0      0     152   7.8292         1   
1          893       3    0  47.0      

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
from itertools import combinations
from sklearn.metrics import accuracy_score


train_data = file_names_drop[0][1]
test_data = file_names_drop[1][1]
submission_data = pd.read_csv(file_path + 'submission.csv')

'''
select = ["Pclass", "Sex", "SibSp", "Parch"]
drop_select = []

interaction_df = pd.DataFrame()

for l, k in enumerate(select):
    for j in select[l:]:
        if k != j:
            interaction_df[f'{k}_{j}'] = df[k] * df[j]


df = pd.concat([df, interaction_df], axis=1)
df = df.drop(columns = drop_select)


select = list(df.columns)
select.remove('PassengerId',)
print(select)
'''
#select = ['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked', 'Cluster', 'Family', 'Sex_Age', 'Age_Fare']
select = ['Pclass', 'Sex', 'Age', 'Ticket', 'Fare', 'Embarked', 'Cluster']

r = []

for i in range(1, len(select) + 1):
    for subset in combinations(select, i):
        # 建立選定的特徵的 X 和 y
        X = pd.get_dummies(train_data[list(subset)])
        #X = pd.get_dummies(df[select])
        y = train_data['Survived']
        X_test = pd.get_dummies(test_data[list(subset)])

        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)
        Model = RandomForestClassifier(n_estimators=200, random_state=2, max_depth=3)

        Model.fit(X, y)
        y_pred = Model.predict(X_test)

        submission_target = submission_data['Survived']
        accuracy = accuracy_score(submission_target, y_pred)

        feature_importances = Model.feature_importances_
        #print(f"模型對 submission.csv 的準確度: {accuracy:.4f}")
        r.append((subset, accuracy, feature_importances))

# 找到最佳特徵組合
best_features, best_accuracy, feature_importances = max(r, key=lambda x: x[1])
print(f"最佳特徵組合: {best_features}")
print(f"最佳準確度: {best_accuracy:.4f}")

for feature, importance in zip(select, feature_importances):
    print(f"{feature}: {importance:.4f}")

'''
print('\nFeature_Importance')
# 如果你想查看特徵的重要性，可以這樣做
feature_importances = Model.feature_importances_
for feature, importance in zip(features, feature_importances):
    print(f"{feature}: {importance:.4f}")
'''

最佳特徵組合: ('Pclass', 'Sex', 'Ticket')
最佳準確度: 0.7847
Pclass: 0.2231
Sex: 0.5417
Age: 0.2352


'\nprint(\'\nFeature_Importance\')\n# 如果你想查看特徵的重要性，可以這樣做\nfeature_importances = Model.feature_importances_\nfor feature, importance in zip(features, feature_importances):\n    print(f"{feature}: {importance:.4f}")\n'

In [50]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score

# 多個隨機森林做集成學習，round 0.78~0.75
train_data = file_names_drop[0][1]
test_data = file_names_drop[1][1]
submission_data = pd.read_csv(file_path + 'submission.csv')

#select = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']
select = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked', 'Cluster', 'Family', 'Sex_Age', 'Age_Ticket']

select_features1 = ['Pclass', 'Sex', 'Fare']
select_features2 = ['Sex', 'Ticket', 'Fare']
select_features3 = ['Sex', 'Age', 'Ticket', 'Fare']

X1 = pd.get_dummies(train_data[select_features1])
X2 = pd.get_dummies(train_data[select_features2])
X3 = pd.get_dummies(train_data[select_features3])
y = train_data['Survived']

X_test1 = pd.get_dummies(test_data[select_features1])
X_test2 = pd.get_dummies(test_data[select_features2])
X_test3 = pd.get_dummies(test_data[select_features3])


rf1 = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=10)
rf2 = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=10)
rf3 = RandomForestClassifier(n_estimators=200, max_depth=7, random_state=10)


rf1.fit(X1, y)
rf2.fit(X2, y)
rf3.fit(X3, y)

Model = VotingClassifier(
    estimators=[('rf1', rf1), ('rf2', rf2), ('rf3', rf3)],
    voting='soft'  # 'soft' for probabilities; 'hard' for majority voting
)

y_pred1 = rf1.predict_proba(X_test1)
y_pred2 = rf2.predict_proba(X_test2)
y_pred3 = rf3.predict_proba(X_test3)

Model.fit(pd.concat([X1, X2, X3], axis=1), y)
y_pred = Model.predict(pd.concat([X_test1, X_test2, X_test3], axis=1))



submission_target = submission_data['Survived']
accuracy = accuracy_score(submission_target, y_pred)
print(f"模型對 submission.csv 的準確度: {accuracy:.4f}")

print('\n select_features1')
feature_importances = rf1.feature_importances_
for feature, importance in zip(select_features1, feature_importances):
    print(f"{feature}: {importance:.4f}")

print('\n select_features2')
feature_importances = rf2.feature_importances_
for feature, importance in zip(select_features2, feature_importances):
    print(f"{feature}: {importance:.4f}")

print('\n select_features3')
feature_importances = rf3.feature_importances_
for feature, importance in zip(select_features3, feature_importances):
    print(f"{feature}: {importance:.4f}")



模型對 submission.csv 的準確度: 0.7679

 select_features1
Pclass: 0.2107
Sex: 0.5341
Fare: 0.2553

 select_features2
Sex: 0.4769
Ticket: 0.2660
Fare: 0.2571

 select_features3
Sex: 0.3851
Age: 0.1289
Ticket: 0.2545
Fare: 0.2315


In [130]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


train_data = file_names_drop[0][1]
test_data = file_names_drop[1][1]
submission_data = pd.read_csv(file_path + 'submission.csv')

#select = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']
select = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked', 'Cluster', 'Family', 'Sex_Age', 'Age_Fare']
select1 = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Family', 'Sex_Age', 'Age_Fare']
select2 = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cluster', 'Family', 'Sex_Age', 'Age_Fare']


X = pd.get_dummies(train_data[list(select)])
X1 = pd.get_dummies(train_data[list(select1)])
X2 = pd.get_dummies(train_data[list(select2)])
y = train_data['Survived']
X_test1 = pd.get_dummies(test_data[list(select1)])
X_test2 = pd.get_dummies(test_data[list(select2)])

rf_model = RandomForestClassifier(n_estimators=200, random_state=2, max_depth=2)
dt_model = DecisionTreeClassifier(max_depth=3, random_state=2)

rf_model.fit(X1, y)
dt_model.fit(X2, y)

y_pred_rf = rf_model.predict_proba(X_test1)[:, 1]
y_pred_dt = dt_model.predict_proba(X_test2)[:, 1]


# 最終加權結果 (可以調整權重 0.5 和 0.5)
y_pred = (0.5 * y_pred_rf + 0.5 * y_pred_dt) > 0.5


submission_target = submission_data['Survived']
accuracy = accuracy_score(submission_target, y_pred)
print(f"模型對 submission.csv 的準確度: {accuracy:.4f}")


print('\n RandomForest')
feature_importances = rf_model.feature_importances_
for feature, importance in zip(select1, feature_importances):
    print(f"{feature}: {importance:.4f}")

print('\n DecisionTree')
feature_importances = dt_model.feature_importances_
for feature, importance in zip(select2, feature_importances):
    print(f"{feature}: {importance:.4f}")


模型對 submission.csv 的準確度: 0.7703

 RandomForest
Pclass: 0.1243
Sex: 0.3564
Age: 0.0241
SibSp: 0.0067
Parch: 0.0123
Ticket: 0.0457
Fare: 0.1250
Family: 0.0197
Sex_Age: 0.2304
Age_Fare: 0.0555

 DecisionTree
Pclass: 0.1678
Sex: 0.6701
Age: 0.0000
SibSp: 0.0000
Parch: 0.0000
Ticket: 0.0600
Fare: 0.0785
Cluster: 0.0000
Family: 0.0000
Sex_Age: 0.0000
Age_Fare: 0.0236


In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
from itertools import combinations

train_data = file_names_drop[0][1]
test_data = file_names_drop[1][1]
submission_data = pd.read_csv(file_path + 'submission.csv')

# 使用必要的特徵進行建模
select_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Embarked', 'Family', 'Sex_Age', 'Age_Fare',
       'Ticket_Count']  # 替換成你要使用的特徵

y = train_data['Survived']
submission_target = submission_data['Survived']

# 設定模型和記錄最佳特徵
best_accuracy = 0
best_features = []

# 將特徵隨機組合進行模型訓練
for i in range(1, len(select_features) + 1):
    feature_combinations = list(combinations(select_features, i))

    for subset in feature_combinations:
        X_train = pd.get_dummies(train_data[list(subset)])
        X_test = pd.get_dummies(test_data[list(subset)])

        # 初始化模型
        rf_model = RandomForestClassifier(n_estimators=200, random_state=2, max_depth=3)
        lr_model = LogisticRegression(random_state=2, max_iter=500)
        nb_model = GaussianNB()

        ensemble_model = VotingClassifier(
            estimators=[('rf', rf_model), ('lr', lr_model), ('nb', nb_model)],
            voting='soft'
        )

        # 訓練集成模型
        ensemble_model.fit(X_train, y)
        y_pred = ensemble_model.predict(X_test)

        # 計算準確度
        accuracy = accuracy_score(submission_target, y_pred)

        # 如果此組合表現最佳，更新最佳組合和準確率
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = subset

print(f"最佳特徵組合: {best_features}")
print(f"最佳準確度: {best_accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c